In [1]:
from Plan import CreateSamples_renewed
from Prepare import OT2Commands as ALH
from Prepare import OT2Graphing as ographing
from opentrons import simulate, execute, protocol_api
import importlib

In [2]:
importlib.reload(CreateSamples_renewed)
importlib.reload(ALH)

<module 'Prepare.OT2Commands' from 'C:\\Users\\Edwin\\Desktop\\OT2-DOE\\PlanPrepareProcess\\Prepare\\OT2Commands.py'>

In [28]:
path = r"Testing Plans\04_28_21_CdSe_Ligand_Study_Trial.csv"
chem_path = r"Chemical Database.csv"
plan = CreateSamples_renewed.get_experiment_plan(path, chem_path)
plan

{'Sample Amount': 5,
 'Sample Unit': 'mL',
 'Component Shorthand Names': ['CdAc', 'Se', 'OleicAc', 'Oleylamine'],
 'Component Concentration Units': ['molarity',
  'molarity',
  'molarity',
  'molarity'],
 'Component Concentration Linspaces [min, max, n]': [[0.0625, 0.0625, 1],
  [0.25, 0.25, 1],
  [0, 0.5, 2],
  [0, 0.25, 2]],
 'Stock Names': ['CdAc-TOP-stock',
  'Se-TOP-stock',
  'OleicAc-stock',
  'Oleylamine-stock'],
 'Stock Concentration Units': ['molarity', 'molarity', 'volf', 'volf'],
 'Stock Concentrations': [0.5, 1, 1, 1],
 'OT2 Destination Labwares': ['fischer_24_wellplate_7400ul',
  'fischer_24_wellplate_7400ul',
  'fischer_24_wellplate_7400ul',
  'fischer_24_wellplate_7400ul'],
 'OT2 Destination Labware Slots': [2, 3, 5, 6],
 'OT2 Sample Labware Start': 0,
 'OT2 Stock Labwares': ['20mlscintillation_12_wellplate_18000ul',
  '20mlscintillation_12_wellplate_18000ul'],
 'OT2 Stock Labware Slots': ['1', '4'],
 'OT2 Stock Labware Stock Order': [1, 2, 3, 4, 5],
 'OT2 Left Pipette':

In [31]:
concentration_df = CreateSamples_renewed.concentration_from_linspace(plan['Component Shorthand Names'], 
                                                                     plan['Component Concentration Linspaces [min, max, n]'],
                                                                     plan['Component Concentration Units'],
                                                                     unity_filter=False)
concentration_df

,CdAc concentration molarity,Se concentration molarity,OleicAc concentration molarity,Oleylamine concentration molarity
0,0.0625,0.25,0.0,0.00
1,0.0625,0.25,0.0,0.25
2,0.0625,0.25,0.5,0.00
3,0.0625,0.25,0.5,0.25


In [34]:
complete_amounts_added = CreateSamples_renewed.determine_component_amounts(plan, concentration_df, nan_fill_value=0)
stock_dict = CreateSamples_renewed.stock_dictionary(plan['Stock Names'], plan['Stock Concentration Units'], plan['Stock Concentrations'])
complete_df = CreateSamples_renewed.calculate_stock_volumes_from_component_concs(plan, complete_amounts_added, stock_dict)
complete_df = CreateSamples_renewed.complete_missing_volume_with_commmon_solvent(plan['Sample Amount'], complete_df, stock_dict, 'Octadecene')

You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units


In [35]:
stock_volumes = CreateSamples_renewed.convert_mL_to_uL(complete_df)
stock_volumes

,CdAc concentration molarity,Se concentration molarity,OleicAc concentration molarity,Oleylamine concentration molarity,CdAc amount mass g,CdAc amount volume uL,Se amount mass g,Se amount volume uL,OleicAc amount mass g,OleicAc amount volume uL,Oleylamine amount mass g,Oleylamine amount volume uL,CdAc-TOP-stock amount volume uL,Se-TOP-stock amount volume uL,OleicAc-stock amount volume uL,Oleylamine-stock amount volume uL,Octadecene-stock volume uL
0,0.0625,0.25,0.0,0.00,0.072031,30.782585,0.0987,20.519751,0.00000,0.000000,0.000000,0.000000,625.0,1250.0,0.000000,0.000000,3125.000000
1,0.0625,0.25,0.0,0.25,0.072031,30.782585,0.0987,20.519751,0.00000,0.000000,0.334363,411.269988,625.0,1250.0,0.000000,411.269988,2713.730012
2,0.0625,0.25,0.5,0.00,0.072031,30.782585,0.0987,20.519751,0.70615,793.426966,0.000000,0.000000,625.0,1250.0,793.426966,0.000000,2331.573034
3,0.0625,0.25,0.5,0.25,0.072031,30.782585,0.0987,20.519751,0.70615,793.426966,0.334363,411.269988,625.0,1250.0,793.426966,411.269988,1920.303046


In [36]:
labware_dir_path = r"Custom Labware"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)
protocol = simulate.get_protocol_api('2.3', extra_labware=custom_labware_dict) # make note this must be run constantly ~ if say making loaded dict twice
loaded_dict = ALH.loading_labware(protocol, plan)

C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults


In [37]:
max_source_vol = 18000 # if the list index reports out of range most likely not enough slots aka too much volume
stock_position_info = ALH.stock_well_ranges(stock_volumes, loaded_dict, max_source_vol) 
stock_position_info

{'CdAc-TOP-stock amount volume uL': {'Ranges': [[0, 4]],
  'Stock Wells': [A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'Se-TOP-stock amount volume uL': {'Ranges': [[0, 4]],
  'Stock Wells': [A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'OleicAc-stock amount volume uL': {'Ranges': [[0, 4]],
  'Stock Wells': [A3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'Oleylamine-stock amount volume uL': {'Ranges': [[0, 4]],
  'Stock Wells': [A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'Octadecene-stock volume uL': {'Ranges': [[0, 4]],
  'Stock Wells': [B1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]}}

In [38]:
directions = ALH.create_sample_making_directions(stock_volumes, stock_position_info, loaded_dict) 
# these directions should be used to make the df, unless there is a final move??

In [39]:
ALH.pipette_volumes_sample_wise(protocol, directions, loaded_dict)

Picking up tip from A1 of Opentrons 96 Tip Rack 1000 µL on 11
Transferring 625.0 from A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A1 of Fischer 24 Well Plate 7400 ÂµL on 2
Aspirating 625.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Dispensing 625.0 uL into A1 of Fischer 24 Well Plate 7400 ÂµL on 2 at 1000.0 uL/sec
Returning tip
Dropping tip into A1 of Opentrons 96 Tip Rack 1000 µL on 11
Picking up tip from A2 of Opentrons 96 Tip Rack 1000 µL on 11
Transferring 1250.0 from A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A1 of Fischer 24 Well Plate 7400 ÂµL on 2
Aspirating 625.0 uL from A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Dispensing 625.0 uL into A1 of Fischer 24 Well Plate 7400 ÂµL on 2 at 1000.0 uL/sec
Aspirating 625.0 uL from A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Dispensing 625.0 uL into A1 of Fischer 24 Well Plate 7400 ÂµL on 2 at 1000.0 uL/sec
Returning tip
Dropping 